In [1]:
import requests
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_players = pd.read_parquet('df_players.parquet')
df_positions = pd.read_parquet('df_positions.parquet')
df_teams = pd.read_parquet('df_teams.parquet')

In [2]:
# Supposons que df_players est votre DataFrame préparé
# Il doit contenir les colonnes : 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent'

# Exemple initialisation ou import réel:
# df_players = pd.read_csv('votre_fichier.csv')
# df_players['selected_by_percent'] = df_players['selected_by_percent'].astype(float)

budget = 1500  # Exemple budget total (en 0.1M £)
alpha = 0    # Pondération ownership (à ajuster: plus grand = plus pénalisant)

df_players = df_players.reset_index(drop=True)  # Indices alignés

# Créer problème d'optimisation
prob = LpProblem("FPL_Team_Optimizer", LpMaximize)

# Variables binaires pour chaque joueur
player_vars = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(df_players))]

# Fonction objectif : maximiser points_per_cost tout en minimisant selected_by_percent
prob += lpSum([
    df_players.loc[i, 'points_per_cost'] * player_vars[i] - 
    alpha * df_players.loc[i, 'selected_by_percent'] * player_vars[i]
    for i in range(len(df_players))
])

# Contraintes d’effectif
prob += lpSum(player_vars) == 15
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'GKP']) == 2
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'DEF']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'MID']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'FWD']) == 3

# Contrainte : max 3 joueurs par équipe
for club in df_players['team_name'].unique():
    prob += lpSum([
        player_vars[i]
        for i in range(len(df_players))
        if df_players.loc[i, 'team_name'] == club
    ]) <= 3

# Contraintes budget
prob += lpSum([df_players.loc[i, 'price'] * player_vars[i] for i in range(len(df_players))]) <= budget

# Contrainte minimum minutes (exemple)
prob += lpSum([df_players.loc[i, 'minutes'] * player_vars[i] for i in range(len(df_players))]) >= 1000

# Résoudre
prob.solve()

# Extraire liste des joueurs sélectionnés
selected_players = [df_players.loc[i] for i in range(len(df_players)) if player_vars[i].varValue == 1]
selected_df = pd.DataFrame(selected_players)

# Afficher résultats clés
print(selected_df[['web_name', 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent']])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.12.1/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/a773f415ede24af383178bbe92d9e65f-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/a773f415ede24af383178bbe92d9e65f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 32 COLUMNS
At line 5302 RHS
At line 5330 BOUNDS
At line 6072 ENDATA
Problem MODEL has 27 rows, 741 columns and 3378 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 8.99571 - 0.00 seconds
Cgl0004I processed model has 27 rows, 617 columns (617 integer (544 of which binary)) and 2882 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 2.22045e-16
Cbc0038I Solution found of -8.99571
Cbc0038I Cleaned solution of -8.99571
Cbc0038I Before mini branch and bound, 617 integers at bound fixed and 0 continu

In [3]:
selected_df.sort_values(by = ['position'])

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,points_per_minutes,fdr_next_6,ict_index,selected_by_percent,selected_rank,form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,assists,goals_scored,goal_involvements,expected_goals,expected_assists,expected_goal_involvements,GI_on_xGI,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90
0,Marcos,Senesi,BOU,Bournemouth,DEF,48.0,37.0,0.770833,0.082222,3.166667,20.2,16.5,22,8.7,450.0,1891122,684816,166601,22297,1.0,0.0,1.0,0.14,0.15,0.29,3.448276,0.03,0.0,0.03,0.06,0.72,1.00,13.20
1,Marc,Guéhi,CRY,Crystal Palace,DEF,47.0,36.0,0.765957,0.080000,3.333333,18.0,25.1,12,9.3,450.0,1831703,289957,377021,91562,1.0,1.0,2.0,0.39,0.04,0.43,4.651163,0.08,0.0,0.01,0.09,0.94,0.40,7.20
2,Trevoh,Chalobah,CHE,Chelsea,DEF,52.0,38.0,0.730769,0.088785,2.833333,29.1,10.7,41,8.0,428.0,1193186,247105,236980,42954,1.0,2.0,3.0,0.75,0.55,1.30,2.307692,0.16,0.0,0.12,0.28,1.10,1.05,10.72
4,Dan,Burn,NEW,Newcastle,DEF,50.0,32.0,0.640000,0.071111,3.000000,13.6,5.9,63,7.0,450.0,377773,65082,161035,35954,0.0,0.0,0.0,0.05,0.12,0.17,0.000000,0.01,0.0,0.02,0.03,0.53,0.60,9.00
7,Riccardo,Calafiori,ARS,Arsenal,DEF,57.0,35.0,0.614035,0.091864,2.666667,22.6,14.4,27,3.0,381.0,1529973,134358,318930,68654,2.0,1.0,3.0,1.57,0.09,1.66,1.807229,0.37,0.0,0.02,0.39,0.60,0.47,4.72
37,Wilson,Isidor,SUN,Sunderland,FWD,55.0,25.0,0.454545,0.106383,3.000000,22.7,2.3,122,6.3,235.0,291390,149656,70315,26339,0.0,3.0,3.0,1.01,0.00,1.01,2.970297,0.39,0.0,0.00,0.39,1.25,1.15,3.83
40,João Pedro,João Pedro,CHE,Chelsea,FWD,78.0,35.0,0.448718,0.083333,2.833333,36.3,67.8,1,6.0,420.0,2569589,277452,965482,207045,3.0,2.0,5.0,1.50,0.20,1.70,2.941176,0.32,0.0,0.04,0.36,1.05,0.86,4.71
46,Richarlison,Richarlison,TOT,Spurs,FWD,68.0,29.0,0.426471,0.083573,2.666667,38.0,16.5,23,4.0,347.0,2018906,456636,772897,97538,1.0,3.0,4.0,1.64,0.18,1.82,2.197802,0.43,0.0,0.05,0.48,1.34,0.78,5.19
3,Robin,Roefs,SUN,Sunderland,GKP,45.0,29.0,0.644444,0.064444,3.000000,13.0,2.6,110,6.7,450.0,290434,64552,83377,16595,0.0,0.0,0.0,0.00,0.00,0.00,NaN,0.00,2.8,0.00,0.00,1.05,0.80,0.00
5,Nick,Pope,NEW,Newcastle,GKP,50.0,32.0,0.640000,0.071111,3.000000,8.8,6.6,58,6.3,450.0,452902,112832,180186,44157,1.0,0.0,1.0,0.00,0.00,0.00,0.000000,0.00,2.0,0.00,0.00,0.53,0.60,0.00


In [4]:
selected_df['price'].sum()

np.float64(838.0)

In [5]:
selected_df['selected_by_percent'].sum()

np.float64(236.70000000000005)